In [2]:
import argparse
import os
import numpy as np
import json
from voc import parse_voc_annotation
from yolo import create_yolov3_model, dummy_loss
from generator import BatchGenerator
from utils.utils import normalize, evaluate, makedirs
from keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
from keras.optimizers import Adam
from callbacks import CustomModelCheckpoint, CustomTensorBoard
from utils.multi_gpu_model import multi_gpu_model
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras
from keras.models import load_model

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
min_input_size =    288
max_input_size=448
anchors =              [55,69, 75,234, 133,240, 136,129, 142,363, 203,290, 228,184, 285,359, 341,260]
labels=['reccoon']
train_times=          8
batch_size=   16
learning_rate=      1e-4
nb_epochs=            100
warmup_epochs=       3
ignore_thresh=  0.5
gpus=   0,1
grid_scales =       [1,1,1]
obj_scale=            5
noobj_scale=          1
xywh_scale=          1
class_scale =        1
saved_weights_name='reccoon.weights'

In [4]:
config = tf.compat.v1.ConfigProto(
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.9)
    # device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [5]:

def create_training_instances(
    train_annot_folder,
    train_image_folder,
    train_cache,
    valid_annot_folder,
    valid_image_folder,valid_cache,labels
):
        train_ints,train_labels = parse_voc_annotation(train_annot_folder, train_image_folder,train_cache,labels)
        print(train_labels)
    # parse annotations of the validation set, if any, otherwise split the training set
        valid_ints, valid_labels = parse_voc_annotation(valid_annot_folder,valid_image_folder,valid_cache,labels)
        
        if len(labels) > 0:
                overlap_labels = set(labels).intersection(set(train_labels.keys()))

                print('Seen labels: \t'  + str(train_labels) + '\n')
                print('Given labels: \t' + str(labels))

        # return None, None, None if some given label is not in the dataset
                if len(overlap_labels) < len(labels):
                     print('Some labels have no annotations! Please revise the list of labels in the config.json.')
                     return None, None, None
        else:
             print('No labels are provided. Train on all seen labels.')
             print(train_labels)
             labels = train_labels.keys()
        
        max_box_per_image=max([len(inst['object']) for inst in (train_ints + valid_ints)])
        return train_ints, valid_ints, sorted(labels),max_box_per_image   
        
        

In [6]:
def create_callbacks(saved_weights_name, tensorboard_logs, model_to_save):
    makedirs(tensorboard_logs)
    
    early_stop = EarlyStopping(
        monitor     = 'loss', 
        min_delta   = 0.01, 
        patience    = 7, 
        mode        = 'min', 
        verbose     = 1
    )
   
    return [early_stop]


'''
   checkpoint = ModelCheckpoint(
        
        filepath        = 'reccoon.h5',# + '{epoch:02d}.h5', 
        monitor         = 'loss', 
        verbose         = 1, 
        save_best_only  = True, 
        mode            = 'min', 
        period          = 1
    )
    reduce_on_plateau = ReduceLROnPlateau(
        monitor  = 'loss',
        factor   = 0.1,
        patience = 2,
        verbose  = 1,
        mode     = 'min',
        epsilon  = 0.01,
        cooldown = 0,
        min_lr   = 0)  
'''
     


"\n   checkpoint = ModelCheckpoint(\n        \n        filepath        = 'reccoon.h5',# + '{epoch:02d}.h5', \n        monitor         = 'loss', \n        verbose         = 1, \n        save_best_only  = True, \n        mode            = 'min', \n        period          = 1\n    )\n    reduce_on_plateau = ReduceLROnPlateau(\n        monitor  = 'loss',\n        factor   = 0.1,\n        patience = 2,\n        verbose  = 1,\n        mode     = 'min',\n        epsilon  = 0.01,\n        cooldown = 0,\n        min_lr   = 0)  \n"

In [7]:
def create_model(
    nb_class, 
    anchors, 
    max_box_per_image, 
    max_grid, batch_size, 
    warmup_batches, 
    ignore_thresh, 
  
    saved_weights_name, 
    lr,
    grid_scales,
    obj_scale,
    noobj_scale,
    xywh_scale,
    class_scale  
):
     template_model, infer_model = create_yolov3_model(
            nb_class            = nb_class, 
            anchors             = anchors, 
            max_box_per_image   = max_box_per_image, 
            max_grid            = max_grid, 
            batch_size          = batch_size, 
            warmup_batches      = warmup_batches,
            ignore_thresh       = ignore_thresh,
            grid_scales         = grid_scales,
            obj_scale           = obj_scale,
            noobj_scale         = noobj_scale,
            xywh_scale          = xywh_scale,
            class_scale         = class_scale
        )  
#loading_weight
     if os.path.exists(saved_weights_name): 
        print("\nLoading pretrained weights.\n")
        template_model.load_weights(saved_weights_name)
     else :
        template_model.load_weights("backend.h5", by_name=True)
     optimizer = Adam(lr=lr, clipnorm=0.001)
     train_model = template_model   
     train_model.compile(loss=dummy_loss, optimizer=optimizer)             

     return train_model, infer_model



In [8]:
train_ints, valid_ints,labels, max_box_per_image = create_training_instances(
        '/home/vivek/machinevision/keras-yolo3-master/dataset/train_annot_folder/',
        '/home/vivek/machinevision/keras-yolo3-master/dataset/train_image_folder/',
        'reccoon_train.pkl',
        '/home/vivek/machinevision/keras-yolo3-master/dataset/val_annot_folder/',
        '/home/vivek/machinevision/keras-yolo3-master/dataset/val_image_folder/',
    'reccoon_val.pkl', ['raccoon'])

{'raccoon': 163}
Seen labels: 	{'raccoon': 163}

Given labels: 	['raccoon']


In [9]:
 print('\nTraining on: \t' + str(labels) + '\n')


Training on: 	['raccoon']



In [10]:
train_generator = BatchGenerator(
        instances           = train_ints, 
        anchors             = anchors,   
        labels              = labels,        
        downsample          = 32, # ratio between network input's size and network output's size, 32 for YOLOv3
        max_box_per_image   = max_box_per_image,
        batch_size          = batch_size,
        min_net_size        = min_input_size,
        max_net_size        = max_input_size ,  
        shuffle             = True, 
        jitter              = 0.3, 
        norm                = normalize)
    
valid_generator = BatchGenerator(
        instances           = valid_ints, 
        anchors             = anchors,   
        labels              = labels,        
        downsample          = 32, # ratio between network input's size and network output's size, 32 for YOLOv3
        max_box_per_image   = max_box_per_image,
        batch_size          = batch_size,
        min_net_size        =min_input_size,
        max_net_size        = max_input_size ,  
        shuffle             = True, 
        jitter              = 0.0, 
        norm                = normalize )

In [11]:
warmup_batches = warmup_epochs *train_times*len(train_generator)
print(warmup_batches)

240


In [12]:
train_model, infer_model = create_model(
        nb_class            = len(labels), 
        anchors             = anchors, 
        max_box_per_image   = max_box_per_image, 
        max_grid            = [max_input_size,max_input_size], 
        batch_size          = batch_size, 
        warmup_batches      = warmup_batches,
        ignore_thresh       = ignore_thresh,
       
        saved_weights_name  = saved_weights_name,
        lr                  = learning_rate,
        grid_scales         = grid_scales,
        obj_scale           = obj_scale,
        noobj_scale         = noobj_scale,
        xywh_scale          = xywh_scale,
        class_scale         = class_scale
    )

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.cast` instead.


In [13]:
callbacks = create_callbacks('backend.h5','logs', infer_model)

In [ ]:
 train_model.fit_generator(
        generator        = train_generator, 
        steps_per_epoch  = len(train_generator) * train_times, 
        epochs           =   2,    #nb_epochs+ warmup_epochs, 
        verbose          = 2 , 
        workers          = 4,
        max_queue_size   = 8,
     callbacks=callbacks
     
    )

In [14]:
infer_model = load_model('raccoon.h5')

/home/vivek/anaconda3/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [15]:
average_precisions = evaluate(infer_model, valid_generator)

In [16]:
for label, average_precision in average_precisions.items():
        print(labels[label] + ': {:.4f}'.format(average_precision))
print('mAP: {:.4f}'.format(sum(average_precisions.values()) / len(average_precisions))) 

raccoon: 0.0003
mAP: 0.0003


In [17]:
import os
import argparse
import json
import cv2
from utils.utils import get_yolo_boxes, makedirs
from utils.bbox import draw_boxes
from keras.models import load_model
from tqdm import tqdm
import numpy as np

In [44]:
image = cv2.imread('asd.jpg')

In [49]:
net_h, net_w = 416, 416 # a multiple of 32, the smaller the faster
obj_thresh, nms_thresh = .52, .45
# predict the bounding boxes
boxes = get_yolo_boxes(infer_model, [image], net_h, net_w,[17,18, 28,24, 36,34, 42,44, 56,51, 72,66, 90,95, 92,154, 139,281], obj_thresh, nms_thresh)[0]

# draw bounding boxes on the image using labels
draw_boxes(image, boxes,labels, obj_thresh)
#[55,69, 75,234, 133,240, 136,129, 142,363, 203,290, 228,184, 285,359, 341,260]

array([[[  8,  23,   9],
        [  2,  17,   3],
        [ 10,  26,   9],
        ...,
        [  0, 113,  44],
        [  2, 107,  44],
        [  3, 104,  42]],

       [[  3,  18,   4],
        [  4,  19,   5],
        [  6,  22,   5],
        ...,
        [  0, 116,  47],
        [  7, 113,  47],
        [  5, 109,  44]],

       [[  2,  15,   1],
        [  2,  15,   1],
        [  8,  23,   9],
        ...,
        [  1, 114,  46],
        [  6, 115,  47],
        [  8, 114,  47]],

       ...,

       [[106, 115, 129],
        [107, 116, 129],
        [107, 115, 128],
        ...,
        [  7,   6,  10],
        [  1,   0,   2],
        [  0,   0,   1]],

       [[106, 115, 128],
        [110, 120, 130],
        [108, 118, 128],
        ...,
        [  6,   5,   9],
        [  2,   1,   3],
        [  2,   1,   3]],

       [[111, 121, 131],
        [116, 126, 136],
        [113, 123, 133],
        ...,
        [  3,   2,   6],
        [  0,   0,   1],
        [  0,   0,   1]]

In [50]:
cv2.imwrite('dfd5.jpeg', np.uint8(image))   


True